# 1.0 LangGraph basics

Derived from [![LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/) 

![Screenshot 2024-08-20 at 3.11.22 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dba5f465f6e9a2482ad935_simple-graph1.png)

In [ ]:
%%capture --no-stderr
%pip install openai~=2.0 --upgrade --quiet
%pip install python-dotenv~=1.0 --upgrade --quiet
%pip install langchain~=0.3 langchain_openai~=0.3 langgraph~=0.6 langchain_community~=0.3.5 --upgrade --quiet

### Load environment variables

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    # Get API key from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    # Load from .env file for local development
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar")
        print("   → Add a secret named 'OPENAI_API_KEY'")
        print("   → Toggle 'Notebook access' to enable it")
    else:
        print("   → Create a .env file with: OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key configured!")

### Setup models

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## Let's get started with LangGraph!

Some key concepts in LangGraph are:
* [Nodes](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes)
* [Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges)
* [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)
* [START Node](https://langchain-ai.github.io/langgraph/concepts/low_level/#start-node)
* [END Node](https://langchain-ai.github.io/langgraph/concepts/low_level/#end-node)
* [Conditional Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges)
* [Graph Compilation](https://langchain-ai.github.io/langgraph/concepts/low_level/#compiling-your-graph)


## State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph. 

The State schema serves as the input schema for all Nodes and Edges in the graph. All nodes are expected to communicate with that schema.

LangGraph offers flexibility in how you define your state schema, accommodating various Python [types](https://docs.python.org/3/library/stdtypes.html#type-objects) and validation approaches!

**The main documented way** to specify the schema of a graph is by using `TypedDict`. However, Pydantic BaseModel and dataclasses are also supported. Read more about **[Pydantic state models here](https://langchain-ai.github.io/langgraph/how-tos/state-model/)**.

### TypedDict
#### _Simply a dictionary with a fixed set of keys and some typing support. At runtime, it’s still a regular dictionary._

We can use the `TypedDict` class from python's `typing` module.

It allows you to specify keys and their corresponding value types.

But, note that these are type hints. They can used by static type checkers (like [mypy](https://github.com/python/mypy)) or IDEs to catch potential type-related errors before the code is run. But they are not enforced at runtime!

```python
from typing_extensions import TypedDict

class TypedDictState(TypedDict):
    foo: str
    bar: str
```

### State with TypedDict
Let's go ahead and use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [ ]:
from typing_extensions import TypedDict, NotRequired

class State(TypedDict):
    graph_state: str # You can also use optional fields like this: `NotRequired[str]`

#### Examples of use

In [ ]:
my_state1: State = {"graph_state": "Hi, this is Zaphod Beeblebrox."}

# Incorrect use
my_state2: State = {"graph_state": "Hi, this is Zaphod Beeblebrox.", "age": 42} # This will result in warning
my_state3: State = {} # This will also result in warning


# Optional fields
class OptionalState(TypedDict):
    graph_state: NotRequired[str]

my_optional_state1: OptionalState = {} # This will NOT result in warning, as `graph_state` is optional ()

## Nodes

[Nodes](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes) are just python functions.

The first positional argument is the state, as defined above.

Because the state is a `TypedDict` with schema as defined above, each node can access the key, `graph_state`, with `state['graph_state']`.

Each node returns a new value of the state key `graph_state`.

By default, the new value returned by each node [will override](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) the prior state value.

In [ ]:
def node_1(state: State) -> State:
    print("---Node 1---")
    return {"graph_state": state['graph_state'] + " I am"}

def node_2(state: State) -> State:
    print("---Node 2---")
    return {"graph_state": state['graph_state'] + " happy!"}

def node_3(state: State) -> State:
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}

## Edges

[Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges) connect the nodes.

Normal Edges are used if you want to *always* go from, for example, `node_1` to `node_2`.

[Conditional Edges](https://langchain-ai.github.io/langgraph/reference/graphs/?h=conditional+edge#langgraph.graph.StateGraph.add_conditional_edges) are used want to *optionally* route between nodes.
 
Conditional edges are implemented as functions that return the next node to visit based upon some logic.

In [ ]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    
    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state'] 
    
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"

## Graph Construction

Now, we build the graph from our [components](
https://langchain-ai.github.io/langgraph/concepts/low_level/) defined above.

The [StateGraph class](https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph) is the graph class that we can use.
 
First, we initialize a StateGraph with the `State` class we defined above.
 
Then, we add our nodes and edges.

We use the [`START` Node, a special node](https://langchain-ai.github.io/langgraph/concepts/low_level/#start-node) that sends user input to the graph, to indicate where to start our graph.
 
The [`END` Node](https://langchain-ai.github.io/langgraph/concepts/low_level/#end-node) is a special node that represents a terminal node. 

Finally, we [compile our graph](https://langchain-ai.github.io/langgraph/concepts/low_level/#compiling-your-graph) to perform a few basic checks on the graph structure. 

We can visualize the graph as a [Mermaid diagram](https://github.com/mermaid-js/mermaid).

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

## Graph Invocation

The compiled graph implements the [runnable](https://python.langchain.com/v0.1/docs/expression_language/interface/) protocol.

This provides a standard way to execute LangChain components. 
 
`invoke` is one of the standard methods in this interface.

The input is a dictionary `{"graph_state": "Hi, this is lance."}`, which sets the initial value for our graph state dict.

When `invoke` is called, the graph starts execution from the `START` node.

It progresses through the defined nodes (`node_1`, `node_2`, `node_3`) in order.

The conditional edge will traverse from node `1` to node `2` or `3` using a 50/50 decision rule. 

Each node function receives the current state and returns a new value, which overrides the graph state.

The execution continues until it reaches the `END` node.

In [ ]:
graph.invoke({"graph_state" : "Hi, this is ToBe."})

`invoke` runs the entire graph synchronously.

This waits for each step to complete before moving to the next.

It returns the final state of the graph after all nodes have executed.

In this case, it returns the state after `node_3` has completed: 

```
{'graph_state': 'Hi, this is Lance. I am sad!'}
```

----

## EXERCISE! Build your own graph

**Task:**
- Create a graph that takes a name as input, converts it into a cooler sounding name and generates a mood.

#### Below is a skeleton for the graph.

In [ ]:
# STATE
class MyState(TypedDict):
    # TODO: Add name and mood
    pass # Remove this line

# NODES
def generate_cool_name(state: MyState) -> MyState:
    print("---Generate Cool Name---")
    initial_mood = "😑"
    new_name = llm.invoke(f"Generate a cool name based on the name {state['name']}. Answer with just the name.").content
    return {} # TODO: Return the new state

def happy_node(state: MyState) -> MyState:
    print("---Happy Node---")
    return {} # TODO: Return the new state

def sad_node(state: MyState) -> MyState:
    print("---Sad Node---")
    return {} # TODO: Return the new state


# EDGES
def decide_mood(state: MyState) -> Literal["happy_node", "sad_node"]:
    name = state['name']
    if len(name) % 2 == 0:
        return "happy_node"
    return "sad_node"


# GRAPH
# Configure nodes
builder = StateGraph(MyState)
builder.add_node("generate_cool_name", generate_cool_name)
# TODO: Add missing nodes
#builder.add_node(...)
#builder.add_node(...)

# Configure edges
builder.add_edge(START, "generate_cool_name")
# TODO: Add missing edges
#builder.add_conditional_edges()
#builder.add_edge(, END)
#builder.add_edge(, END)

# Compile and view
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke
graph.invoke({"name": "ToBe"})